# `rules` are magic functions and regular expressions that express unique displays from strings.

In [1]:
import IPython

In [2]:
from .base import MagicStrings

In [3]:
docstring_pattern = globals().get('docstring_pattern', None)
if not docstring_pattern:
    for i, transformer in enumerate(IPython.get_ipython().input_transformers_cleanup):
        if isinstance(transformer, IPython.core.inputtransformer2.PromptStripper) and transformer.prompt_re.pattern == '^(>>>|\\.\\.\\.)( |$)':
            docstring_pattern = transformer

In [4]:
@MagicStrings.append('*>>> *')
def doctest(str):
    import doctest
    global runner
    runner = doctest.DocTestRunner()
    runner.run(
        doctest.DocTest(
            examples=doctest.DocTestParser().get_examples(str),
            globs=IPython.get_ipython().user_global_ns, 
            name=__name__, 
            filename=__name__, 
            lineno=1, 
            docstring=str)
    )
    return '🏆'*(len(runner.test.examples)-runner.failures)

In [6]:
def load_ipython_extension(ip):
    transformers = list(ip.input_transformers_cleanup)
    for i, transformer in enumerate(transformers):
        if transformer is docstring_pattern:
            ip.input_transformers_cleanup.pop(i)
            break
            
    ip.display_formatter.mimebundle_formatter.for_type(str, MagicStrings)
    
def unload_ipython_extension(ip):
    ip.display_formatter.mimebundle_formatter.for_type(str)
    ip.input_transformers_cleanup.append(docstring_pattern)

    
__name__ == '__main__' and load_ipython_extension(IPython.get_ipython())